In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import globvars as gv
import os
import mne,utils

In [ ]:
#anns_cnv, ivalis_perside = utils.intervalJSON2Anns(rawname_)

In [ ]:
from globvars import gp

In [ ]:
#gv.gen_subj_info
import json

In [ ]:
import utils_preproc as upre

In [ ]:

    # print('\nmy prev interval lengths')
    # lens_cnv = utils.getIntervalsTotalLens(anns_cnv, True, times=raw.times) 
    # display(lens_cnv )
    # #display(anns_cnv.description)
    # if mts_trem_str not in anns_cnv.description:
    #     print('!! There is no tremor, accdording to prev me')
rawname_ = 'S01_off_hold'
aritfs = ['_ann_MEGartif_flt', '_ann_MEGartif']

lens, ann_dict = upre.getIntervalInfoFromRawname(rawname_, ann_MEGartif_prefix_to_use=aritfs);#,interval=(150,250));
lens_short = upre.getIntervalInfoFromRawname(rawname_,crop=(170,220), ann_MEGartif_prefix_to_use=aritfs);

In [ ]:
sfreq = 1/256
sfreq

In [ ]:
#df.query('type == "artif"')

In [ ]:
list(sorted(lens.keys()))[0].find('_L')

In [ ]:
rawnames = []
rawnames +=  ['S06_off_hold', 'S06_off_move'] + ['S06_on_hold', 'S06_on_move'] 
rawnames +=  ['S07_off_hold', 'S07_off_move'] + ['S07_on_hold', 'S07_on_move'] 
rawnames +=  ['S08_off_rest', 'S08_on_rest'] 

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
upre.getRawnameListStructure(rawnames)

In [ ]:
rawnames =   ['S01_off_hold', 'S01_off_move'] + ['S01_on_hold', 'S01_on_move']
rawnames +=  ['S02_off_hold', 'S02_off_move'] + ['S02_on_hold', 'S02_on_move'] 
rawnames +=  ['S03_off_hold', 'S03_off_move']
rawnames +=  ['S04_off_hold', 'S04_off_move'] + ['S04_on_hold', 'S04_on_move'] 
rawnames +=  ['S05_off_hold', 'S05_off_move'] + ['S05_on_hold', 'S05_on_move'] 
rawnames +=  ['S06_off_hold', 'S06_off_move'] + ['S06_on_hold', 'S06_on_move'] 
rawnames +=  ['S07_off_hold', 'S07_off_move'] + ['S07_on_hold', 'S07_on_move'] 
rawnames +=  ['S08_off_rest', 'S08_on_rest'] 
rawnames +=  ['S09_off_rest'] 
rawnames +=  ['S10_off_rest', 'S10_off_move']

assert 31 == len(rawnames)

In [ ]:
%%capture cap

In [ ]:
for rawname_ in rawnames:
    try:
        upre.getGenIntervalInfoFromRawname(rawname_);
    except FileNotFoundError as fnfe:
        print(fnfe)
        
with open('interval_stats_output.txt', 'w') as f:
    f.write(cap.stdout)

        

# good rawnames

In [ ]:
sfreq = 256

In [ ]:
rawnames =   ['S01_off_hold', 'S01_off_move'] + ['S01_on_hold', 'S01_on_move']
rawnames +=  ['S02_off_hold', 'S02_off_move'] + ['S02_on_hold', 'S02_on_move'] 
rawnames +=  ['S03_off_hold', 'S03_off_move']
rawnames +=  ['S04_off_hold', 'S04_off_move'] + ['S04_on_hold', 'S04_on_move'] 
rawnames +=  ['S05_off_hold', 'S05_off_move'] + ['S05_on_hold', 'S05_on_move'] 
#rawnames +=  ['S06_off_hold', 'S06_off_move'] + ['S06_on_hold', 'S06_on_move'] 
rawnames +=  ['S07_off_hold', 'S07_off_move'] + ['S07_on_hold', 'S07_on_move'] 
#rawnames +=  ['S08_off_rest', 'S08_on_rest'] 
#rawnames +=  ['S09_off_rest'] 
#rawnames +=  ['S10_off_rest', 'S10_off_move']
from globvars import gp
len(rawnames)


In [ ]:
dfs = []
for rawname_ in rawnames:
    # from srun_resave_resample.sh, needed to add '_ann_' prefix
    artifs = '_ann_MEGartif,_ann_MEGartif_muscle,_ann_MEGartif_ICA'.split(',')
    #aritfs = ['MEGartif_muscle', '_ann_MEGartif_flt', '_ann_MEGartif', '_ann_MEGartif_ICA']

    lens, ann_dict = upre.getIntervalInfoFromRawname(rawname_, 
        ann_MEGartif_prefix_to_use=artifs);#,interval=(150,250));    
    
    tmin,tmax = 0,-1
    for anntype,anns in ann_dict.items():
        print(anntype,anns)
        tmin = min( tmin, np.min(anns.onset) )
        tmax = max(tmax, np.max(anns.onset + anns.duration))
    print( tmin,tmax )
    # create bins, set them for anns, remove artif outside anns, calc lens
    
    subj = rawname_.split('_')[0]
    maintremside = gv.gen_subj_info[subj]['tremor_side']
    move_side = gv.gen_subj_info[subj].get('move_side','UNDEF')
    tremfreq = gv.gen_subj_info[subj]['tremfreq']

    import pandas as pd    
    nbins = np.ceil(tmax * sfreq).astype(int); print('nbins = ',nbins)
    df = pd.DataFrame({'bin':np.arange(nbins), 'type':['_']*nbins })
    df['rawname'] = rawname_
    df = df.set_index(['rawname','bin'])
    #df['side'] = '_'
    df['move_sidelet'] = move_side[0].upper()
    df['subject'] = subj
    #df['sidelet'] = '_'
    df['medcond'] = rawname_.split('_')[1]
    task = rawname_.split('_')[2]
    df['task'] = task
    for it in gp.int_types_basic_sided:
        df[it] = False
    for anntype,anns in ann_dict.items():
        for an in anns:
            #print(anntype,an)
            end = an["onset"] + an["duration"]
            end = end * sfreq
            inds = df.query(f'bin >= {an["onset"] * sfreq} and bin < {end}').index
            #df.loc[inds, 'type'] = anntype
            desc = an["description"]
            df.loc[inds, desc] = True
            
            if desc.startswith(task):
                if desc[-1] == move_side[0].upper():
                    df.loc[inds, f'{task}_moveside'] = True
                else:
                    df.loc[inds, f'{task}_moveopside'] = True
            #df.loc[inds, 'sidelet'] = an["description"].split('_')[-1]
    dfs += [df]

df2 = pd.concat(dfs)

In [ ]:
df2.columns

In [ ]:
# decorate
for col in ['move_moveside', 'move_moveopside']:
    df2.loc[ df2[col].isna(), col ] = False
    df2[col] = df2[col].astype(bool)
    
# todo rename o 
for behstate in ['trem','notrem']:
    df2[f'{behstate}_moveside'] = False
    c = (df2['move_sidelet'] == 'R') & df2[f'{behstate}_R']
    df2.loc[c,f'{behstate}_moveside'] = True
    c = (df2['move_sidelet'] == 'L') & df2[f'{behstate}_L']
    df2.loc[c,f'{behstate}_moveside'] = True

    df2[f'{behstate}_moveopside'] = False
    c = (df2['move_sidelet'] == 'L') & df2[f'{behstate}_R']
    df2.loc[c,f'{behstate}_moveopside'] = True
    c = (df2['move_sidelet'] == 'R') & df2[f'{behstate}_L']
    df2.loc[c,f'{behstate}_moveopside'] = True

df2['trem_moveopside_during_move'] = df2['trem_moveopside'] & df2['move_moveside']
df2['trem_moveopside_during_hold'] = df2['trem_moveopside'] & df2['hold_moveside']
for behstate in ['trem','notrem','move','hold']:
    df2[f'{behstate}_bothsides'] = df2[f'{behstate}_R'] & df2[f'{behstate}_L']
#df2['notrem_bothsides'] = df2['notrem_R'] & df2['notrem_L']

artif_cols = [col for col in df2.columns if col.startswith('BAD')]
print('artif_cols =',artif_cols)
df2['some_artif'] = df2[artif_cols[0]]
for col in artif_cols[1:]:
    df2['some_artif'] |= df2[col]    

In [ ]:
weird_states = 'movetrem_L, holdtrem_L, movetrem_R, holdtrem_R, undef_L, undef_R, subtrem_L, subtrem_R'.split(', ')
df2['some_weird_state_moveside'] = False
for col in weird_states:
    if col not in df2:
        print('skip due to nonpresent ',col)
        continue
    c = df2['move_sidelet'] == col[-1]
    c = c & (~df2[col].isna())
    df2.loc[c, 'some_weird_state_moveside'] |= df2.loc[c, col]
    
# set types
for artif_col in artif_cols + ['some_artif','some_weird_state_moveside']:
    df2.loc[ df2[artif_col].isna(), artif_col] = False
    df2[artif_col] = df2[artif_col].astype(bool)
    
grp0 = df2.groupby(['subject','medcond'])

In [ ]:
df2.columns

In [ ]:
print('Proportion of diff artifacts (pct)')
dfr.div( grp0.size(), axis=0 ) * 100  # just dfr/grp.size() does not work

In [ ]:
# length of diff artif in seconds
dfr = df2.groupby(['subject','medcond'])[artif_cols + ['some_artif','some_weird_state_moveside']].sum() / sfreq
display(dfr)

In [ ]:
for behstate in ['trem','move','notrem']:
    print(f'{behstate} duration (espciall move both sides together)')
    dfr = grp0[[f'{behstate}_moveside', f'{behstate}_moveopside',f'{behstate}_bothsides']].sum() / sfreq
    display(dfr)

In [ ]:
# for all points (inc unlabeled and artifacted)
def f(dftmp):
    return dftmp[['trem_moveside','trem_moveopside','trem_bothsides',
                  'move_moveside','trem_moveopside_during_move',
                 'hold_moveside', 'trem_moveopside_during_hold']].astype(float).sum() / sfreq
dfr = grp0.apply(f)
dfr['trem_bothsides_ratio'] = dfr['trem_bothsides'] / dfr['trem_moveside']
display(dfr)
dfr.reset_index().groupby('medcond')['trem_bothsides_ratio'].describe() * 100

In [ ]:
# duration per dataset
df2.groupby(['subject','medcond','task']).size() / sfreq

In [ ]:
# duration per medcond
grp0.size() / sfreq

# Tremor stats for non-artif points

In [ ]:
df2_.columns

In [ ]:
# for non-artif points
df2_ = df2.query('(not some_artif) and (not some_weird_state_moveside)')
print('Nonartif len {}, all len {}'.format(len(df2_), len(df2)) )
prop_nonartif = df2_.groupby(['subject','medcond']).size() / grp0.size()
print('Porportion nonartif')
display(prop_nonartif)

In [ ]:
# All beh states (no artif and wierd states)
def f(dftmp):
    return dftmp[['trem_moveside','notrem_moveside', 'hold_moveside','move_moveside']].astype(float).sum() / sfreq
dfr = df2_.groupby(['subject','medcond']).apply(f)

# Step 1: Reset index
dfr = dfr.reset_index()
# Step 2: Pivot dataframe
dfr = dfr.pivot(index='subject', columns='medcond')
# Step 3: Swap levels of columns
#dfr.columns = dfr.columns.swaplevel(0, 1)
# Step 4: Join levels of columns with underscore
dfr.columns = dfr.columns.map('_'.join).str.replace('moveside_','')

mask = dfr.columns.str.endswith('_off')
# Select 'off' and 'on' columns using mask
dfr_off = dfr.loc[:, mask]
dfr_on = dfr.loc[:, ~mask]
# Concatenate 'off' and 'on' columns
dfr = pd.concat([dfr_off, dfr_on], axis=1)

# Print new dataframe
display(dfr)
#print('trem_bothsides_ratio (%): Mean over subjects within medcond')
#descr = dfr.reset_index().groupby('medcond')['trem_bothsides_ratio'].describe() * 100
#display(descr)
dfr.describe().loc[['mean','std']] 

In [ ]:
# Tremor on move side only (no artif and wierd states)
def f(dftmp):
    return dftmp[['trem_moveside']].astype(float).sum() 
dfr = df2_.query('trem_moveside and ~trem_moveopside').groupby(['subject','medcond']).apply(f)
dfr = dfr.div( df2_.groupby(['subject','medcond'])['trem_moveside'].sum(), 0 ) * 100
#dfr['trem_only_moveside'] = dfr['trem_moveside'] / dfr['trem_moveside']

# Step 1: Reset index
dfr = dfr.reset_index()
# Step 2: Pivot dataframe
dfr = dfr.pivot(index='subject', columns='medcond')
# Step 3: Swap levels of columns
#dfr.columns = dfr.columns.swaplevel(0, 1)
# Step 4: Join levels of columns with underscore
dfr.columns = dfr.columns.map('_'.join).str.replace('moveside_','')

mask = dfr.columns.str.endswith('_off')
# Select 'off' and 'on' columns using mask
dfr_off = dfr.loc[:, mask]
dfr_on = dfr.loc[:, ~mask]
# Concatenate 'off' and 'on' columns
dfr = pd.concat([dfr_off, dfr_on], axis=1)

# Print new dataframe
display(dfr)
#print('trem_bothsides_ratio (%): Mean over subjects within medcond')
#descr = dfr.reset_index().groupby('medcond')['trem_bothsides_ratio'].describe() * 100
#display(descr)
descr = dfr.describe().loc[['mean','std']] 
display(descr)

means = descr.loc['mean']
stds = descr.loc['std']
s = ('If selecting epochs where tremor appeared exclusively on the movement-performing hand then: '
    'the average percentage with respect to the total move hand side tremor was ')
s += '{:.0f}% (std = {:.0f}%) in OFF condition'.format(means['trem_off'],stds['trem_off'] )  
s += ' and '
s += '{:.0f}% (std = {:.0f}%) in ON condition'.format(means['trem_on'],stds['trem_on'])  
s+='.'
print(s)

In [ ]:
descr

In [ ]:
#proportion of doubleside tremor (no artif and wierd states)
def f(dftmp):
    return dftmp[['trem_moveside','trem_moveopside','trem_bothsides']].astype(float).sum() / sfreq
dfr = df2_.groupby(['subject','medcond']).apply(f)
dfr['trem_bothsides_ratio'] = dfr['trem_bothsides'] / dfr['trem_moveside']
display(dfr)
print('trem_bothsides_ratio (%): Mean over subjects within medcond')
descr = dfr.reset_index().groupby('medcond')['trem_bothsides_ratio'].describe() * 100
display(descr)

### to put in manuscript

In [ ]:
means = descr.to_dict()['mean']
stds = descr.to_dict()['std']
s = ('In our data tremor appeared on both sides of the hand simultaneosuly rather frequently: '
    'the average percentage of bimanual tremor with respect to the move hand side tremor was ')
s += '{:.0f}% (std = {:.0f}%) in OFF condition'.format(means['off'],stds['off'] )  
s += ' and '
s += '{:.0f}% (std = {:.0f}%) in ON condition'.format(means['on'],stds['on'])  
s+='.'
print(s)
